# vnstock_pro_demo — Công cụ phân tích VNStock (chi tiết)

Notebook này tải dữ liệu từ `vnstock`, tính EMA(9,26,100,200) và RSI(9,45), phát hiện **phân kỳ (divergence)** giữa giá và RSI, đánh dấu các điểm **EMA cross / breakpoints**, và vẽ biểu đồ rõ ràng kèm nhận xét ngắn. 

Hướng dẫn: sửa `symbol`, `start_date`, `end_date` ở ô `# ===== USER INPUTS =====` rồi chạy **Run All**.

> Ghi chú: nếu kernel báo lỗi import `vnstock`, hãy mở Terminal và chạy `pip install vnstock pandas numpy matplotlib scipy` rồi restart kernel.

In [ ]:
# ====== Kiểm tra / cài đặt phụ thuộc (chạy nếu thiếu) ======
import importlib, sys, subprocess
required = ['vnstock', 'pandas', 'numpy', 'matplotlib', 'scipy']
missing = [p for p in required if importlib.util.find_spec(p) is None]
if missing:
    print('Thiếu thư viện:', missing)
    print('Cài đặt... (có thể tốn vài phút)')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--quiet'] + missing)
    print('Đã cài xong. Vui lòng restart kernel nếu cần.')
else:
    print('Môi trường OK — tất cả thư viện cần thiết đã có sẵn.')


In [ ]:
# ===== USER INPUTS =====
symbol = 'VNINDEX'  # đổi mã tại đây (vd: 'PDR', 'VIC')
start_date = ''     # định dạng YYYY-MM-DD; để trống để lấy 1 năm gần nhất
end_date = ''       # để trống để lấy tới hôm nay

from datetime import datetime, timedelta
if end_date == '':
    end_date = datetime.today().strftime('%Y-%m-%d')
if start_date == '':
    start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
print(f'Load {symbol} từ {start_date} đến {end_date}')


In [ ]:
# ===== Fetch dữ liệu: thử nhiều API của vnstock để đảm bảo tương thích =====
import pandas as pd
import numpy as np
from datetime import datetime
df = None
try:
    from vnstock import Quote
    q = Quote(symbol=symbol, source='TCBS')
    df = q.history(start=start_date, end=end_date, interval='1D')
    if isinstance(df, dict) and 'data' in df:
        df = pd.DataFrame(df['data'])
    print('Dữ liệu lấy bằng vnstock.Quote.history()')
except Exception as e:
    print('Không lấy được bằng Quote:', e)
    try:
        from vnstock import Vnstock
        stock = Vnstock().stock(symbol=symbol, source='TCBS')
        df = stock.quote.history(start=start_date, end=end_date, interval='1D')
        print('Dữ liệu lấy bằng Vnstock().stock().quote.history()')
    except Exception as e2:
        print('Không lấy bằng Vnstock:', e2)
        try:
            from vnstock import stock_historical_data
            df = stock_historical_data(symbol=symbol, start_date=start_date, end_date=end_date, resolution='1D')
            print('Dữ liệu lấy bằng stock_historical_data()')
        except Exception as e3:
            print('Không thể lấy dữ liệu tự động bằng vnstock trên môi trường này.')
            raise RuntimeError('Vui lòng kiểm tra package vnstock hoặc cài phiên bản tương thích.')

if df is None or len(df) == 0:
    raise RuntimeError('Không có dữ liệu trả về cho symbol này.')
if not isinstance(df, pd.DataFrame):
    df = pd.DataFrame(df)
df.columns = [c.lower() for c in df.columns]
if 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'])
elif 'date' in df.columns:
    df['time'] = pd.to_datetime(df['date'])
else:
    try:
        df.index = pd.to_datetime(df.index)
        df = df.reset_index().rename(columns={'index':'time'})
    except Exception:
        raise RuntimeError('Không tìm được cột thời gian trong dữ liệu.')
df = df.set_index('time')
required_cols = ['open','high','low','close','volume']
available = [c for c in required_cols if c in df.columns]
print('Cột có sẵn:', available)
if 'close' not in df.columns:
    raise RuntimeError('Dữ liệu không có cột close.')
df = df.sort_index()
df.head()


In [ ]:
# ===== Tính EMA và RSI (Wilder smoothing cho RSI) =====
def ema(series, span):
    return series.ewm(span=span, adjust=False).mean()
def rsi_wilder(series, length=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    ma_up = up.ewm(alpha=1/length, adjust=False).mean()
    ma_down = down.ewm(alpha=1/length, adjust=False).mean()
    rs = ma_up / ma_down
    rsi = 100 - (100 / (1 + rs))
    return rsi
df['EMA9'] = ema(df['close'], 9)
df['EMA26'] = ema(df['close'], 26)
df['EMA100'] = ema(df['close'], 100)
df['EMA200'] = ema(df['close'], 200)
df['RSI9'] = rsi_wilder(df['close'], 9)
df['RSI45'] = rsi_wilder(df['close'], 45)
df[['close','EMA9','EMA26','EMA100','EMA200','RSI9','RSI45']].tail()


In [ ]:
# ===== Phát hiện EMA crosses (EMA9 vs EMA26) và breakpoint cross EMA100/EMA200 =====
df['ema9_above_26'] = (df['EMA9'] > df['EMA26']).astype(int)
df['ema9_above_26_prev'] = df['ema9_above_26'].shift(1)
df['ema_cross_up'] = (df['ema9_above_26'] == 1) & (df['ema9_above_26_prev'] == 0)
df['ema_cross_down'] = (df['ema9_above_26'] == 0) & (df['ema9_above_26_prev'] == 1)
df['cross_above_100'] = (df['close'] > df['EMA100']) & (df['close'].shift(1) <= df['EMA100'].shift(1))
df['cross_below_100'] = (df['close'] < df['EMA100']) & (df['close'].shift(1) >= df['EMA100'].shift(1))
df[['close','EMA9','EMA26','EMA100','ema_cross_up','ema_cross_down']].tail()


In [ ]:
# ===== Phát hiện local peaks/troughs để dùng cho phân kỳ =====
from scipy.signal import argrelextrema
def find_peaks_troughs(series, order=5):
    local_max_idx = argrelextrema(series.values, np.greater_equal, order=order)[0]
    local_min_idx = argrelextrema(series.values, np.less_equal, order=order)[0]
    return local_max_idx, local_min_idx
order = 5
max_idx, min_idx = find_peaks_troughs(df['close'], order=order)
print('Số đỉnh', len(max_idx), 'Số đáy', len(min_idx))


In [ ]:
# ===== Tìm divergence: so sánh cặp peak trước và sau (hoặc trough trước và sau) =====
divergences = []  # (date, type, price, rsi_value, comment)
dates = df.index
rsi = df['RSI9']
for i in range(len(max_idx)-1):
    i1 = max_idx[i]
    i2 = max_idx[i+1]
    p1 = df['close'].iloc[i1]
    p2 = df['close'].iloc[i2]
    r1 = rsi.iloc[i1]
    r2 = rsi.iloc[i2]
    if p2 > p1 and r2 < r1:
        divergences.append((dates[i2], 'bearish', float(p2), float(r2), f'Price HH (p1={p1:.2f},p2={p2:.2f}) RSI LH (r1={r1:.1f},r2={r2:.1f})'))
for i in range(len(min_idx)-1):
    i1 = min_idx[i]
    i2 = min_idx[i+1]
    p1 = df['close'].iloc[i1]
    p2 = df['close'].iloc[i2]
    r1 = rsi.iloc[i1]
    r2 = rsi.iloc[i2]
    if p2 < p1 and r2 > r1:
        divergences.append((dates[i2], 'bullish', float(p2), float(r2), f'Price LL (p1={p1:.2f},p2={p2:.2f}) RSI HL (r1={r1:.1f},r2={r2:.1f})'))
print('Tìm được', len(divergences), 'divergences.')


In [ ]:
# ===== Vẽ biểu đồ =====
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
plt.style.use('seaborn-whitegrid')
fig, axes = plt.subplots(3,1, figsize=(14,12), sharex=True, gridspec_kw={'height_ratios':[3,1,1]})
ax0, ax1, ax2 = axes
ax0.plot(df.index, df['close'], color='black', label='Close')
ax0.plot(df.index, df['EMA9'], color='#1f77b4', label='EMA9')
ax0.plot(df.index, df['EMA26'], color='#ff7f0e', label='EMA26')
ax0.plot(df.index, df['EMA100'], color='#9467bd', label='EMA100')
ax0.plot(df.index, df['EMA200'], color='#d62728', label='EMA200')
ax0.set_title(f'{symbol} — Price + EMA (9/26/100/200)')
ax0.scatter(df.index[df['ema_cross_up']], df['close'][df['ema_cross_up']], marker='^', color='green', s=110, zorder=5, label='EMA9↑EMA26')
ax0.scatter(df.index[df['ema_cross_down']], df['close'][df['ema_cross_down']], marker='v', color='red', s=110, zorder=5, label='EMA9↓EMA26')
ax0.scatter(df.index[df['cross_above_100']], df['close'][df['cross_above_100']], marker='o', color='blue', s=90, zorder=5, label='Cross above EMA100')
ax0.scatter(df.index[df['cross_below_100']], df['close'][df['cross_below_100']], marker='x', color='magenta', s=90, zorder=5, label='Cross below EMA100')
for dt, typ, price, rvalue, comment in divergences:
    if typ == 'bearish':
        ax0.scatter([dt], [price], color='red', s=140, edgecolor='k', zorder=6)
    else:
        ax0.scatter([dt], [price], color='green', s=140, edgecolor='k', zorder=6)
ax0.legend(loc='upper left')
ax1.bar(df.index, df['volume'].fillna(0), color='gray')
ax1.set_ylabel('Volume')
ax2.plot(df.index, df['RSI9'], label='RSI9', color='tab:blue')
ax2.plot(df.index, df['RSI45'], label='RSI45', color='tab:green')
ax2.axhline(70, color='red', ls='--', lw=0.8)
ax2.axhline(30, color='green', ls='--', lw=0.8)
ax2.set_ylabel('RSI')
ax2.legend(loc='upper left')
recent_divs = [d for d in divergences if (df.index[-1] - d[0]).days <= 60]
for dt, typ, price, rvalue, comment in recent_divs:
    ax2.annotate(f'{typ} {comment}', xy=(dt, rvalue), xytext=(dt, rvalue+8 if typ=='bullish' else rvalue-8),
                 arrowprops=dict(arrowstyle='->', color='black'), fontsize=10, bbox=dict(boxstyle='round', fc='w'))
date_fmt = DateFormatter('%Y-%m-%d')
ax2.xaxis.set_major_formatter(date_fmt)
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()


In [ ]:
# ===== Tạo nhận xét tự động ngắn gọn (summary) =====
summary = []
last = df.iloc[-1]
summary.append(f'Giá hiện tại: {last.close:.2f}')
summary.append(f'RSI9: {last.RSI9:.1f}, RSI45: {last.RSI45:.1f}')
if df['ema_cross_up'].iloc[-1]:
    summary.append('EMA9 vừa cắt lên EMA26 (bullish crossover).')
elif df['ema_cross_down'].iloc[-1]:
    summary.append('EMA9 vừa cắt xuống EMA26 (bearish crossover).')
else:
    summary.append('Không có crossover EMA9/26 gần nhất.')
if any(d[1]=='bearish' for d in recent_divs):
    summary.append('Cảnh báo: xuất hiện phân kỳ giảm (bearish divergence) gần đây — thận trọng).')
if any(d[1]=='bullish' for d in recent_divs):
    summary.append('Quan sát: phân kỳ tăng (bullish divergence) — có thể là điểm mua tốt khi có pullback).')
print('\n'.join(summary))


In [ ]:
# ===== Export báo cáo PDF (bao gồm figure) =====
from matplotlib.backends.backend_pdf import PdfPages
pdf_file = f'report_{symbol}_{start_date}_to_{end_date}.pdf'
with PdfPages(pdf_file) as pdf:
    pdf.savefig(fig)
print('Saved PDF report to', pdf_file)


### Hướng dẫn dùng thêm
- Thay `symbol`, `start_date`, `end_date` → Run All để phân tích mã khác.
- Tham số `order` trong `find_peaks_troughs()` điều khiển mức nhạy của việc tìm đỉnh/đáy.
- Nếu API vnstock thay đổi, chỉnh phần `fetch` (mình để nhiều lựa chọn dự phòng).
